In [1]:
# Install all required packages
%package install bloomberg.bquant.signal_lab=1.5.2

Running: micromamba install bloomberg.bquant.signal_lab=1.5.2 --yes --quiet --log-level=error

Note: Packages not from Bloomberg channels are not vetted by Bloomberg.
Please restart the Jupyter kernel if you run into any issues after installing or updating packages via %package.



In [2]:
import bloomberg.bquant.signal_lab as signal_lab
signal_lab.__version__

'1.5.2'

In [3]:
import bql


from bloomberg.bquant.signal_lab.workflow.node import (
    industry_grouping, portfolio_construction)
from bloomberg.bquant.signal_lab.signal.transformers import WeightingScheme
from bloomberg.bquant.signal_lab.workflow.factory import (
    UniverseFactory,
    DataItemFactory,
    SignalFactory,
)
from bloomberg.bquant.signal_lab.workflow import (
    AnalyticsDataConfig,
    build_backtest,
)

from bloomberg.bquant.signal_lab.workflow.utils import get_sandbox_path

import numpy as np
import pandas as pd

In [4]:
# Get the saved DataPack path
bq = bql.Service()
data_pack_path = f"{get_sandbox_path()}/esl/datapack_snapshot"
data_pack_path

's3://awmgd-prod-finml-sandbox-user/bclarke16/esl/datapack_snapshot'

In [5]:
import backtest_params as bp
import importlib

In [6]:
# Key backtest parameters
start = "2020-01-05"
end = "2024-03-01"
universe_name = "INDU Index"

In [7]:
universe, benchmark = bp.get_universe_params(start, end, universe_name, data_pack_path)
price, cur_mkt_cap = bp.get_return_params(start, end, data_pack_path)

In [8]:
# Import the test data set while developing
df = pd.read_csv('test_analysis.csv')

In [9]:
df = df[['Date','Security','Decision','Confidence']]

In [10]:
df

,Date,Security,Decision,Confidence
0,2020-02-06,MMM UN Equity,BUY,70.0
1,2020-02-12,CSCO UW Equity,BUY,80.0
2,2020-02-12,AMGN UQ Equity,BUY,80.0
3,2020-02-13,AXP UN Equity,BUY,80.0
4,2020-02-13,NVDA UQ Equity,BUY,80.0
...,...,...,...,...
891,2025-01-31,XOM UN Equity,BUY,70.0
892,2025-02-04,PFE UN Equity,BUY,70.0
893,2025-02-04,MRK UN Equity,BUY,80.0
894,2025-02-04,AMGN UW Equity,BUY,70.0


In [12]:
def convert_to_figi(df: pd.DataFrame) -> pd.DataFrame:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ = df['Security'].to_list()
    field =  {'figi': bq.data.id_bb_global()}
    req = bql.Request(univ, field)
    data = bq.execute(req)
    figi = data[0].df()
    return df.merge(figi, left_on='Security', right_index=True).sort_index()
    

In [13]:
df1 = convert_to_figi(df)

In [14]:
df1

,Date,Security,Decision,Confidence,figi
0,2020-02-06,MMM UN Equity,BUY,70.0,BBG000BP5522
1,2020-02-12,CSCO UW Equity,BUY,80.0,BBG000C3JBN9
2,2020-02-12,AMGN UQ Equity,BUY,80.0,BBG000BBS644
3,2020-02-13,AXP UN Equity,BUY,80.0,BBG000BCR153
4,2020-02-13,NVDA UQ Equity,BUY,80.0,BBG000BBJSW5
...,...,...,...,...,...
891,2025-01-31,XOM UN Equity,BUY,70.0,BBG000GZQBJ1
892,2025-02-04,PFE UN Equity,BUY,70.0,BBG000BR2F10
893,2025-02-04,MRK UN Equity,BUY,80.0,BBG000BPD257
894,2025-02-04,AMGN UW Equity,BUY,70.0,BBG000BBSCX9


In [ ]:
def event_backtest(df_events: pd.DataFrame):
    pass